In [22]:
def mex(reachable: list[int]) -> int:
    """
    Compute the minimum excludant (mex) of a set of numbers.

    :param reachable: set of integers
    :return: int, minimum non-negative integer not in the set
    """
    mex_value = 0
    while mex_value in reachable:
        mex_value += 1
    return mex_value


def compute_sprague_grundy(
        game_type: str,
        S: list[int],
        max_n: int
        ) -> list[int]:
    """
    Compute Sprague-Grundy values for a given game type (SUBTRACTION or ALLBUT)
        and set S up to max_n.

    :param game_type: str, either "SUBTRACTION" or "ALLBUT"
    :param S: set of allowed moves
    :param max_n: int, maximum number to compute values for
    :return: list of Sprague-Grundy values from 0 to max_n
    """
    grundy = [0] * (max_n + 1)

    for n in range(1, max_n + 1):
        reachable = set()
        
        if game_type == "SUBTRACTION":
            reachable = {
                grundy[n - move]
                for move in S
                if n - move >= 0
                }
        elif game_type == "ALLBUT":
            reachable = {
                grundy[n - move]
                for move in range(1, n + 1)
                if move not in S and n - move >= 0
                }
        else:
            msg = "Invalid game type. Choose 'SUBTRACTION' or 'ALLBUT'."
            raise ValueError(msg)

        grundy[n] = mex(reachable)

    return grundy


def find_period(grundy_values: list[int]) -> tuple[int, int | None]:
    """
    Find the period and pre-period of the Sprague-Grundy values.

    :param grundy_values: list of integers representing the Sprague-Grundy vals
    :return: tuple (pre_period, period) where:
             - pre_period is the length before periodicity starts
             - period is the length of the repeating cycle
    """
    n = len(grundy_values)

    for pre_period in range(n):
        for period in range(1, n - pre_period):
            is_periodic = True
            for i in range(pre_period, n - period):
                if grundy_values[i] != grundy_values[i + period]:
                    is_periodic = False
                    break

            if is_periodic:
                return pre_period, period

    return len(grundy_values), None


def analyze_periodicity(
        grundy_values: list[int]
        ) -> tuple[int, int | None, int | None]:
    """
    Analyze the periodicity and pre-period of Sprague-Grundy values.

    :param grundy_values: list of Sprague-Grundy values
    :return: tuple (pre_period, period, saltus), where:
             - pre_period is the length before periodicity starts
             - period is the length of the repeating cycle
             - saltus is the arithmetic difference in the periodic sequence
                (if applicable)
    """
    n = len(grundy_values)
    
    for pre_period in range(n):
        for period in range(1, n - pre_period):
            if (
                grundy_values[pre_period:pre_period + period]
                == grundy_values[pre_period + period:pre_period + 2 * period]
            ):
                saltus = None
                is_arithmetic = True

                for i in range(pre_period, n - period):
                    if (
                        (grundy_values[i] + grundy_values[i + period])
                        != grundy_values[i]
                    ):
                        is_arithmetic = False
                        break

                if is_arithmetic:
                    saltus = (
                        grundy_values[pre_period + period]
                        - grundy_values[pre_period]
                    )
                
                return pre_period, period, saltus

    return len(grundy_values), None, None


# Task 1 - SUBTRACTION(S)

In [43]:
to_check = (
    {
        "S": {1, 4, 6},
        "max_n": 15,
        "period_len": 5,
        "period": (0, 1, 0, 1, 2)
    },
    {
        "S": {7, 4, 6},
        "max_n": 100,
        "period_len": 11,
        # "period": (0, 1, 0, 1, 2)
    },
    {
        "S": {7, 6, 8},
        "max_n": 100,
        "period_len": 14,
        # "period": (0, 1, 0, 1, 2)
    },
    {
        "S": {7, 8, 10},
        "max_n": 100,
        "period_len": 17,
        # "period": (0, 1, 0, 1, 2)
    },
    {
        "S": {7, 10, 12},
        "max_n": 100,
        "period_len": 19,
        # "period": (0, 1, 0, 1, 2)
    },
    {
        "S": {7, 12, 14},
        "max_n": 100,
        "period_len": 21,
        # "period": (0, 1, 0, 1, 2)
    },
)


print("** Analyzing SUBTRACTION(S) **\n")
for record in to_check:
    S = record["S"]
    max_n = record["max_n"]

    grundy_values = compute_sprague_grundy("SUBTRACTION", S, max_n)
    pre_period, period_len = find_period(grundy_values)

    if "period_len" in record:
        expected_period_len = record["period_len"]
        assert expected_period_len == period_len
        assert grundy_values[:period_len] == grundy_values[:expected_period_len]

    if "period" in record:
        expected_period = record["period"]
        assert tuple(expected_period) == tuple(grundy_values[:period_len])

    print(f"S: {S}")
    # print("\tAll numbers:", grundy_values)
    print(f"\tPeriod len: {period_len: 3} Period: {grundy_values[:period_len]}")
    print()


** Analyzing SUBTRACTION(S) **

S: {1, 4, 6}
	Period len:   5 Period: [0, 1, 0, 1, 2]

S: {4, 6, 7}
	Period len:  11 Period: [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2]

S: {8, 6, 7}
	Period len:  14 Period: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2]

S: {8, 10, 7}
	Period len:  17 Period: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]

S: {10, 12, 7}
	Period len:  19 Period: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]

S: {12, 14, 7}
	Period len:  21 Period: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]



# Task 2 - ALLBUT(S)

In [ ]:
# grundy_values = compute_sprague_grundy("ALLBUT", S, max_n)
# pre_period, period, saltus = analyze_periodicity(grundy_values)
# print(f"Sprague-Grundy values: {grundy_values}")
# print(f"Pre-period: {pre_period}, Period: {period}, Saltus: {saltus}")


In [ ]:
# to_check = (
#     {
#         "S": {1, 4, 6},
#         "max_n": 15,
#         "period_len": 5,
#         "period": (0, 1, 0, 1, 2)
#     },
#     {
#         "S": {7, 4, 6},
#         "max_n": 100,
#         "period_len": 11,
#         # "period": (0, 1, 0, 1, 2)
#     },
#     {
#         "S": {7, 6, 8},
#         "max_n": 100,
#         "period_len": 14,
#         # "period": (0, 1, 0, 1, 2)
#     },
#     {
#         "S": {7, 8, 10},
#         "max_n": 100,
#         "period_len": 17,
#         # "period": (0, 1, 0, 1, 2)
#     },
#     {
#         "S": {7, 10, 12},
#         "max_n": 100,
#         "period_len": 19,
#         # "period": (0, 1, 0, 1, 2)
#     },
#     {
#         "S": {7, 12, 14},
#         "max_n": 100,
#         "period_len": 21,
#         # "period": (0, 1, 0, 1, 2)
#     },
# )


# print("** Analyzing ALLBUT(S) **\n")
# for record in to_check:
#     S = record["S"]
#     max_n = record["max_n"]

#     grundy_values = compute_sprague_grundy("ALLBUT", S, max_n)
#     pre_period, period_len = find_period(grundy_values)

#     if "period_len" in record:
#         expected_period_len = record["period_len"]
#         assert expected_period_len == period_len
#         assert grundy_values[:period_len] == grundy_values[:expected_period_len]

#     if "period" in record:
#         expected_period = record["period"]
#         assert tuple(expected_period) == tuple(grundy_values[:period_len])

#     print(f"S: {S}")
#     # print("\tAll numbers:", grundy_values)
#     print(f"\tPeriod len: {period_len: 3} Period: {grundy_values[:period_len]}")
#     print()
